# Laboratorio 8 
    - Francis Aguilar - 22243
    - César López - 22535
    - Angela García -22869 


enlace al repositorio: https://github.com/angelargd8/lab8-ds


In [1]:
import pandas as pd

# Cargar el archivo
excel_path = "/opt/app/working_dir/principales.xlsx"
xls = pd.ExcelFile(excel_path)

# Ver nombres de las hojas
print(xls.sheet_names)


['Portada-Presentación', 'Índice', 'Directorio ', 'Procedimiento', 'cuadro 1', 'cuadro 2', 'cuadro 3', 'cuadro 4', 'cuadro 5', 'cuadro 6', 'cuadro 7', 'cuadro 8', 'cuadro 9', 'cuadro 10', 'cuadro 11', 'cuadro 12', 'cuadro 13', 'cuadro 14', 'cuadro 15', 'cuadro 16', 'cuadro 17', 'cuadro 18', 'cuadro 19', 'cuadro 20', 'cuadro 21', 'cuadro 22', 'cuadro 23', 'cuadro 24', 'cuadro 25', 'cuadro 26', 'cuadro 27', 'cuadro 28', 'cuadro 29', 'cuadro 30', 'cuadro 31', 'cuadro 32', 'cuadro 33', 'cuadro 34', 'cuadro 35', 'cuadro 36', 'cuadro 37', 'cuadro 38', 'cuadro 39', 'cuadro 40', 'cuadro 41', 'cuadro 42', 'cuadro 43', 'cuadro 44', 'cuadro 45', 'cuadro 46', 'cuadro 47', 'cuadro 48', 'cuadro 49', 'cuadro 50', 'cuadro 51', 'cuadro 52', 'cuadro 53', 'cuadro 54', 'cuadro 55', 'cuadro 56', 'cuadro 57', 'cuadro 58', 'cuadro 59', 'cuadro 60', 'cuadro 61', 'cuadro 62', 'cuadro 63', 'cuadro 64', 'cuadro 65', 'Nota técnica', 'Ficha técnica']


In [97]:
import pandas as pd

excel_path = "/opt/app/working_dir/principales.xlsx"


header_list = ["departamento", "vehículo", "vehiculo", "mes de ocurrencia","día de la semana", "hora de ocurrencia","condición del conductor y sexo","grupos de edad", "tipo de vehículo y sexo de la persona", "zona de ocurrencia", "tipo de accidente y sexo"  ,"tipo", "clase", "marca", "modelo de vehículo", "sexo", "grupo", "edad","tipo de vehículo", "clases de vehículos", "color de vehículo" ]

def read_cuadro(sheet_num, categoria):
    sheet = f"cuadro {sheet_num}"
    raw = pd.read_excel(excel_path, sheet_name=sheet, header=None)

    header_idx = None

    for i, row in enumerate(raw.values):
        row = raw.iloc[i].astype(str).str.lower()
        if any(word in row.tolist() for word in header_list):
            header_idx = i
            break

    if header_idx is None:
        return None

    # Tomo la fila de encabezados
    header_row = raw.iloc[header_idx].tolist()
    # Tomo la fila siguiente (por si son los años / meses / días)
    next_row = raw.iloc[header_idx + 1].tolist()

    # Combinar: si la celda en header_row está vacía, usar la de next_row
    combined_header = []
    for h, n in zip(header_row, next_row):
        if pd.isna(h) or str(h).strip() == "":
            combined_header.append(n)
        elif str(h).lower() in ["año de ocurrencia", "mes de ocurrencia", "día de la semana", "total","hombre","mujer","ignorado","grupos de edad"]:
            combined_header.append(n)  # usar el detalle
        else:
            combined_header.append(h)

    # Extraer datos desde dos filas después
    df = raw.iloc[header_idx + 2:].copy()
    df.columns = combined_header

    # Quitar filas vacías y ruido
    df = df.dropna(how="all")
    df = df[~df.iloc[:,0].astype(str).str.contains("fuente|nota|cuadro|serie", case=False, na=False)]

    # Limpiar nombres de columnas
    df.columns = [str(c).strip() for c in df.columns]

    # Agregar metadata
    df["categoria"] = categoria
    df["cuadro"] = sheet_num

    # --- Limpiar filas basura ---
    # Quitar filas con primera columna vacía
    df = df[df.iloc[:,0].notna() & (df.iloc[:,0].astype(str).str.strip() != "")]
    
    # Quitar filas completamente vacías en todas las columnas excepto metadata
    cols_sin_meta = df.columns.difference(["categoria", "cuadro"])
    df = df.dropna(subset=cols_sin_meta, how="all")
    
    return df

def build_dict(rango, categoria):
    cuadros = {}
    for i in rango:
        try:
            df = read_cuadro(i, categoria)
            if df is not None:
                cuadros[f"{categoria}_{i}"] = df
        except Exception as e:
            print(f"Error en cuadro {i}: {e}")
    return cuadros

# Procesar
hechos_dfs = build_dict(range(1, 17), "hechos")
vehiculos_dfs  = build_dict(range(17, 29), "vehiculos")
lesionados_dfs = build_dict(range(31, 47), "lesionados")
fallecidos_dfs = build_dict(range(47, 63), "fallecidos")

# Verificar cuáles cuadros se cargaron
print(list(hechos_dfs.keys()))
print(list(vehiculos_dfs.keys()))
print(list(lesionados_dfs.keys()))
print(list(fallecidos_dfs.keys()))

# mostrar el primero 
print(hechos_dfs["hechos_1"].head())
#print(vehiculos_dfs["vehiculos_17"].head())
#print(lesionados_dfs["lesionados_32"].head())
#print(fallecidos_dfs["fallecidos_48"].head())


    

['hechos_1', 'hechos_2', 'hechos_3', 'hechos_4', 'hechos_5', 'hechos_6', 'hechos_7', 'hechos_8', 'hechos_9', 'hechos_11', 'hechos_12', 'hechos_13', 'hechos_14', 'hechos_15', 'hechos_16']
['vehiculos_17', 'vehiculos_18', 'vehiculos_19', 'vehiculos_20', 'vehiculos_21', 'vehiculos_22', 'vehiculos_23', 'vehiculos_24', 'vehiculos_25', 'vehiculos_26', 'vehiculos_27', 'vehiculos_28']
['lesionados_31', 'lesionados_32', 'lesionados_33', 'lesionados_34', 'lesionados_35', 'lesionados_36', 'lesionados_37', 'lesionados_38', 'lesionados_39', 'lesionados_40', 'lesionados_41', 'lesionados_42', 'lesionados_43', 'lesionados_44', 'lesionados_45', 'lesionados_46']
['fallecidos_47', 'fallecidos_48', 'fallecidos_49', 'fallecidos_50', 'fallecidos_51', 'fallecidos_52', 'fallecidos_53', 'fallecidos_54', 'fallecidos_55', 'fallecidos_56', 'fallecidos_57', 'fallecidos_58', 'fallecidos_59', 'fallecidos_60', 'fallecidos_61', 'fallecidos_62']
     Departamento  2020  2021.0  2022.0  2023.0  2024.0  nan categoria  cu

In [98]:
#hechos_dfs["hechos_1"].to_csv("hechos1.csv", index=False, encoding="utf-8")
##print(vehiculos_dfs["vehiculos_18"].head())
#vehiculos_dfs["vehiculos_18"].to_csv("vehiculos_18.csv", index=False, encoding="utf-8")
#fallecidos_dfs["fallecidos_48"].to_csv("fallecidos_48.csv", index=False, encoding="utf-8")
#lesionados_dfs["lesionados_40"].to_csv("lesionados_40.csv", index=False, encoding="utf-8")

In [96]:
from pyspark.sql import SparkSession

# creacion de la aplicacion
spark = SparkSession.getActiveSession() or SparkSession.builder.appName("MiApp").getOrCreate()
sc = spark.sparkContext

1. Mostrar cuántos registros hay en cada tabla (hechos, vehículos, fallecidos, 
lesionados). Muestre algunos resultados con la función .show(). Genere un describe y 
summary para aquellas columnas que considere importantes según cada archivo.


2. Identificar los años disponibles en cada tabla y validar si coinciden. 

3.  Mostrar los valores distintos de tipo de accidente.

4.  Calcular cuántos departamentos únicos aparecen en las bases. 

5. ¿Cuál es el total de accidentes por año y departamento? Apóyese de la función 
groupBy. Investigue la función display que tiene Databricks y muestre su resultado en 
formato de gráfico de barras. 

6. ¿Qué día de la semana registra más accidentes en 2024? Graficar con display en un 
gráfico de columnas. 

7. Mostrar la distribución de accidentes por hora del día en el municipio de 
Guatemala. Graficar en un histograma. 